In [52]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import os
from scipy.stats import linregress
from pandas_datareader import data as pdr
import yfinance as yf
from glob import glob
yf.pdr_override()

In [24]:

# Initial Scraping of tickers


In [25]:
df = pd.read_csv("nasdaq_screener.csv")
df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$126.05,-0.61,-0.482%,3.723212e+10,United States,1999.0,3138761,Industrials,Electrical Products
1,AA,Alcoa Corporation Common Stock,$34.76,0.85,2.507%,6.201236e+09,United States,2016.0,4171345,Industrials,Aluminum
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,$10.62,0.03,0.283%,7.646090e+08,NaN,2021.0,230915,Industrials,Metal Fabrications
3,AACG,ATA Creativity Global American Depositary Shares,$1.37,0.02,1.481%,4.332237e+07,China,2008.0,5909,Consumer Discretionary,Educational Services
4,AACI,Armada Acquisition Corp. I Common Stock,$10.59,0.02,0.189%,0.000000e+00,United States,2021.0,54208,Finance,Blank Checks


In [57]:
df_big_enough_corps = df[df['Market Cap'] >= 50_000_000]
print("Nasdaq listed: ", len(df), "\nBig Enough (> 50mil market cap):", len(df_big_enough_corps))

Nasdaq listed:  7592 
Big Enough (> 50mil market cap): 4951


In [62]:
ticker_list = df_big_enough_corps[~df_big_enough_corps['Symbol'].isnull()]['Symbol'].to_list() # there was a nan in there for some reason

ticker_list = list(map(lambda x: x.replace('/', '-'), ticker_list))
start, end = dt.datetime(2003, 1, 1), dt.datetime.now()
for i, ticker in enumerate(ticker_list):
    try:
        stock_history_df = pdr.get_data_yahoo(ticker, start, end)
        print(f"{ticker} ({i} / {len(ticker_list)})")
        # take out weekends, holidays
        index_zeros = stock_history_df[stock_history_df['Volume'] == 0].index
        stock_history_df.drop(index_zeros, inplace=True)
        save_path = os.path.join('TickerData', ticker + '.csv')
        stock_history_df.to_csv(save_path, index=True)
    except Exception as e:
       # By this way we can know about the type of error occurring
        print("The error is: ",e)

[*********************100%***********************]  1 of 1 completed
A (0 / 4951)
[*********************100%***********************]  1 of 1 completed
AA (1 / 4951)
[*********************100%***********************]  1 of 1 completed
AAC (2 / 4951)
[*********************100%***********************]  1 of 1 completed
AADI (3 / 4951)
[*********************100%***********************]  1 of 1 completed
AAIC (4 / 4951)
[*********************100%***********************]  1 of 1 completed
AAIN (5 / 4951)
[*********************100%***********************]  1 of 1 completed
AAL (6 / 4951)
[*********************100%***********************]  1 of 1 completed
AAMC (7 / 4951)
[*********************100%***********************]  1 of 1 completed
AAN (8 / 4951)
[*********************100%***********************]  1 of 1 completed
AAOI (9 / 4951)
[*********************100%***********************]  1 of 1 completed
AAON (10 / 4951)
[*********************100%***********************]  1 of 1 completed
AAP


1 Failed download:
['RCA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2003-01-01 00:00:00 -> 2023-07-30 01:31:21.731905)')



RCA (3683 / 4951)
[*********************100%***********************]  1 of 1 completed
RCAT (3684 / 4951)
[*********************100%***********************]  1 of 1 completed
RCB (3685 / 4951)
[*********************100%***********************]  1 of 1 completed
RCC (3686 / 4951)
[*********************100%***********************]  1 of 1 completed
RCEL (3687 / 4951)
[*********************100%***********************]  1 of 1 completed
RCI (3688 / 4951)
[*********************100%***********************]  1 of 1 completed
RCKT (3689 / 4951)
[*********************100%***********************]  1 of 1 completed
RCKY (3690 / 4951)
[*********************100%***********************]  1 of 1 completed
RCL (3691 / 4951)
[*********************100%***********************]  1 of 1 completed
RCM (3692 / 4951)
[*********************100%***********************]  1 of 1 completed
RCMT (3693 / 4951)
[*********************100%***********************]  1 of 1 completed
RCUS (3694 / 4951)
[*****************


1 Failed download:
['SFB']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")



SFB (3970 / 4951)
[*********************100%***********************]  1 of 1 completed
SFBC (3971 / 4951)
[*********************100%***********************]  1 of 1 completed
SFBS (3972 / 4951)
[*********************100%***********************]  1 of 1 completed
SFIX (3973 / 4951)
[*********************100%***********************]  1 of 1 completed
SFL (3974 / 4951)
[*********************100%***********************]  1 of 1 completed
SFM (3975 / 4951)
[*********************100%***********************]  1 of 1 completed
SFNC (3976 / 4951)
[*********************100%***********************]  1 of 1 completed
SFST (3977 / 4951)
[*********************100%***********************]  1 of 1 completed
SFWL (3978 / 4951)
[*********************100%***********************]  1 of 1 completed
SG (3979 / 4951)
[*********************100%***********************]  1 of 1 completed
SGA (3980 / 4951)
[*********************100%***********************]  1 of 1 completed
SGC (3981 / 4951)
[******************

In [59]:
len(glob("./TickerData/*.csv"))

4951